In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
import shutil

#from klusta.launch import klusta as klaunch

In [15]:
# session paths
source  = '/home/sobolev/nevermind/Andrey/data'
animal  = '006865'
session = '006865_hippoSIT_2021-12-09_15-28-36'
session_path = os.path.join(source, animal, session)

In [11]:
#session_path = 'Z:\\Andrey\\sandbox'
#ss_path = os.path.join(session_path, 'ss2')
#animal  = 'GB049'
#session = '2021-07-13T15-12-29'
#filebase = '%s_%s' % (animal, session)

## Copy Neurosuite settings files

In [16]:
settings_source = os.path.join(source, animal)
ns_settings_file = os.path.join(settings_source, 'default.xml')
kwik_params_file = os.path.join(settings_source, 'default.prm')
kwik_probes_file = os.path.join(settings_source, 'default.prb')

shutil.copy(ns_settings_file, os.path.join(session_path, '%s.xml' % session))
shutil.copy(kwik_params_file, os.path.join(session_path, '%s.prm' % session))
shutil.copy(kwik_probes_file, os.path.join(session_path, '%s.prb' % session))

with open(os.path.join(session_path, '%s.prm' % session), 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    lines = "experiment_name = '%s'\nprb_file = '%s.prb'\n \n" % (session, session)
    f.write(lines.rstrip('\r\n') + '\n' + content)

## Conversion of MCS .h5 to .dat file

http://neurosuite.sourceforge.net/formats.html


In [13]:
crossref = None
#crossref = 11

MCS_h5_name = [x for x in os.listdir(session_path) if x.startswith('Andrey') and x.endswith('.h5')][0]
chunk_size = 25000 # 1 second at sampling rate 25kHz
dest_file = os.path.join(session_path, '%s.dat' % session)
if crossref is not None:
    dest_file = os.path.join(session_path, '%s.cf.dat' % session)
if not os.path.exists(session_path):
    os.makedirs(session_path)
if os.path.exists(dest_file):
    os.remove(dest_file)

with h5py.File(os.path.join(session_path, MCS_h5_name), 'r') as f:
    #ds = f['Data']['Recording_0']['AnalogStream']['Stream_1']['ChannelData']
    ds = f['Data']['Recording_0']['AnalogStream']['Stream_2']['ChannelData']
    print("Channels: %s, samples: %s, type: %s" % (ds.shape[0], ds.shape[1], ds.dtype))
    
    chunks_no = int(np.ceil(ds.shape[1]/chunk_size))
    #chunks_no = 60
    for i in range(chunks_no):
        idx = i * chunk_size
        testds = ds[:, idx:idx + chunk_size]
        data = np.array(ds[:, idx:idx + chunk_size]).astype(np.int16)  # int16 is correct
        if crossref is not None:
            to_sub = np.array(data[crossref])
            data -= to_sub
        with open(dest_file, 'ab') as f:
            data.T.flatten().tofile(f)
            
        print("%s percent complete\r" % str(round(100*(i + 1)/float(chunks_no))), end='')

Channels: 32, samples: 48008115, type: int32
